In [1]:
import pandas as pd

__ = {
    "outlook": [
        "sunny", "sunny", "overcast", "rainy", "rainy", "rainy", "overcast",
        "sunny", "sunny", "rainy", "sunny", "overcast", "overcast", "rainy"
    ],
    "temperature": [
        "hot", "hot", "hot", "mild", "cool", "cool", "cool",
        "mild", "cool", "mild", "mild", "mild", "hot", "mild"
    ],
    "humidity": [
        "high", "high", "high", "high", "normal", "normal", "normal",
        "high", "normal", "normal", "normal", "high", "high", "high"
    ],
    "wind": [
        "weak", "strong", "weak", "weak", "weak", "strong", "strong",
        "weak", "weak", "weak", "strong", "strong", "weak", "strong"
    ],
    "play": [
        "no", "no", "yes", "yes", "yes", "no", "yes",
        "no", "yes", "yes", "yes", "yes", "yes", "no"
    ]
}

df = pd.DataFrame(__)
class_att = df[df.columns[-1]]
classes: list[str] = class_att.unique().tolist()

# df, classes, df.columns, class_att.name in df.columns, classes.index("yes")
class_att

0      no
1      no
2     yes
3     yes
4     yes
5      no
6     yes
7      no
8     yes
9     yes
10    yes
11    yes
12    yes
13     no
Name: play, dtype: object

In [2]:
def data_as_dict(df: pd.DataFrame):
    res: dict[str, dict[str, list[int]]] = {}
    class_att = df[df.columns[-1]]
    classes: list[str] = class_att.unique().tolist()
    df = df.drop(columns=df.columns[-1])

    for attcol in df.columns:
        res[attcol] = {}
        for attval, classval in zip(df[attcol], class_att):
            if attval not in res[attcol]:
                res[attcol][attval] = [0] * len(classes)
            res[attcol][attval][classes.index(classval)] += 1
    return res

data_as_dict(df)

{'outlook': {'sunny': [3, 2], 'overcast': [0, 4], 'rainy': [2, 3]},
 'temperature': {'hot': [2, 2], 'mild': [2, 4], 'cool': [1, 3]},
 'humidity': {'high': [4, 4], 'normal': [1, 5]},
 'wind': {'weak': [2, 6], 'strong': [3, 3]}}

In [3]:
def gini_split(att: dict[str, list[int]], total: int):
    gini = 0.0
    for _, counts in att.items():
        subtotal = sum(counts)
        if subtotal == 0:
            continue

        score = 1.0
        for c in counts:
            p = c / subtotal
            score -= p * p

        gini += (subtotal * 1.0 / total) * score
    return gini

def best_split(df: pd.DataFrame):
    data = data_as_dict(df)
    total = len(df)
    best_att = None
    best_gini = float("inf")

    for att, val in data.items():
        gini = gini_split(val, total)
        if gini < best_gini:
            best_gini = gini
            best_att = att
    return best_att, best_gini

In [4]:
root_att, root_gini = best_split(df)
root_att, root_gini

('outlook', 0.34285714285714286)

In [5]:
class DecisionNode:
    def __init__(self, attribute=None, branches=None, label=None, gini=None):
        self.attribute = attribute
        self.branches = branches or {}
        self.label = label
        self.gini = gini
        
    def is_leaf(self):
        return self.label is not None
    
    def __repr__(self, level=0):
        indent = "  " * level
        if self.is_leaf():
            return f"{indent}Leaf: {self.label}\n"
        result = f"{indent}Split on '{self.attribute}' (Gini: {self.gini:.4f})\n"
        for value, child in self.branches.items():
            result += f"{indent}  [{self.attribute}={value}]\n"
            result += child.__repr__(level + 2)
        return result

def build_decision_tree(df: pd.DataFrame, max_depth=None, min_samples_split=2, current_depth=0):
    """
    Build a decision tree using Gini index for splitting.
    
    Parameters:
    - df: DataFrame with features and target (last column is target)
    - max_depth: Maximum depth of the tree (None for no limit)
    - min_samples_split: Minimum samples required to split a node
    - current_depth: Current depth in recursion
    """
    class_att = df[df.columns[-1]]
    if len(class_att.unique()) == 1:
        return DecisionNode(label=class_att.iloc[0])

    if len(df.columns) == 1:
        most_common = class_att.mode()[0]
        return DecisionNode(label=most_common)

    if max_depth is not None and current_depth >= max_depth:
        most_common = class_att.mode()[0]
        return DecisionNode(label=most_common)

    if len(df) < min_samples_split:
        most_common = class_att.mode()[0]
        return DecisionNode(label=most_common)

    best_att, best_gini = best_split(df)
    if best_att is None:
        most_common = class_att.mode()[0]
        return DecisionNode(label=most_common)

    node = DecisionNode(attribute=best_att, gini=best_gini)

    for value in df[best_att].unique():
        subset = df[df[best_att] == value].drop(columns=[best_att])

        if len(subset) > 0:
            node.branches[value] = build_decision_tree(
                subset, 
                max_depth=max_depth, 
                min_samples_split=min_samples_split,
                current_depth=current_depth + 1
            )
        else:
            most_common = class_att.mode()[0]
            node.branches[value] = DecisionNode(label=most_common)
    
    return node

def predict_single(tree: DecisionNode, sample: dict):
    """
    Predict the class for a single sample.
    
    Parameters:
    - tree: Root node of the decision tree
    - sample: Dictionary with feature names and values
    """
    if tree.is_leaf():
        return tree.label

    att_value = sample.get(tree.attribute)
    
    if att_value in tree.branches:
        return predict_single(tree.branches[att_value], sample)
    else:
        return None

def predict(tree: DecisionNode, df: pd.DataFrame):
    """
    Predict classes for multiple samples.
    
    Parameters:
    - tree: Root node of the decision tree
    - df: DataFrame with features (without target column)
    """
    predictions = []
    for _, row in df.iterrows():
        sample = row.to_dict()
        predictions.append(predict_single(tree, sample))
    return predictions

def calculate_accuracy(y_true, y_pred):
    """Calculate accuracy of predictions."""
    correct = sum(1 for true, pred in zip(y_true, y_pred) if true == pred)
    return correct / len(y_true)

tree = build_decision_tree(df, max_depth=5, min_samples_split=2)
print("Decision Tree Structure:")
print(tree)

Decision Tree Structure:
Split on 'outlook' (Gini: 0.3429)
  [outlook=sunny]
    Split on 'humidity' (Gini: 0.0000)
      [humidity=high]
        Leaf: no
      [humidity=normal]
        Leaf: yes
  [outlook=overcast]
    Leaf: yes
  [outlook=rainy]
    Split on 'wind' (Gini: 0.0000)
      [wind=weak]
        Leaf: yes
      [wind=strong]
        Leaf: no



In [6]:
X_train = df.drop(columns=[df.columns[-1]])
y_train = df[df.columns[-1]]

predictions = predict(tree, X_train)
accuracy = calculate_accuracy(y_train, predictions)

print("\nPredictions vs Actual:")
print("-" * 50)
for i, (pred, actual) in enumerate(zip(predictions, y_train)):
    match = "✓" if pred == actual else "✗"
    print(f"Sample {i+1}: Predicted={pred}, Actual={actual} {match}")

print(f"\nAccuracy: {accuracy * 100:.2f}%")

print("\n" + "="*50)
print("Testing with new samples:")
print("="*50)

test_samples = pd.DataFrame([
    {"outlook": "sunny", "temperature": "cool", "humidity": "high", "wind": "weak"},
    {"outlook": "rainy", "temperature": "mild", "humidity": "normal", "wind": "weak"},
    {"outlook": "overcast", "temperature": "hot", "humidity": "high", "wind": "strong"}
])

test_predictions = predict(tree, test_samples)
for i, (idx, row) in enumerate(test_samples.iterrows()):
    print(f"\nSample {i+1}:")
    print(f"  Features: {row.to_dict()}")
    print(f"  Prediction: {test_predictions[i]}")


Predictions vs Actual:
--------------------------------------------------
Sample 1: Predicted=no, Actual=no ✓
Sample 2: Predicted=no, Actual=no ✓
Sample 3: Predicted=yes, Actual=yes ✓
Sample 4: Predicted=yes, Actual=yes ✓
Sample 5: Predicted=yes, Actual=yes ✓
Sample 6: Predicted=no, Actual=no ✓
Sample 7: Predicted=yes, Actual=yes ✓
Sample 8: Predicted=no, Actual=no ✓
Sample 9: Predicted=yes, Actual=yes ✓
Sample 10: Predicted=yes, Actual=yes ✓
Sample 11: Predicted=yes, Actual=yes ✓
Sample 12: Predicted=yes, Actual=yes ✓
Sample 13: Predicted=yes, Actual=yes ✓
Sample 14: Predicted=no, Actual=no ✓

Accuracy: 100.00%

Testing with new samples:

Sample 1:
  Features: {'outlook': 'sunny', 'temperature': 'cool', 'humidity': 'high', 'wind': 'weak'}
  Prediction: no

Sample 2:
  Features: {'outlook': 'rainy', 'temperature': 'mild', 'humidity': 'normal', 'wind': 'weak'}
  Prediction: yes

Sample 3:
  Features: {'outlook': 'overcast', 'temperature': 'hot', 'humidity': 'high', 'wind': 'strong'}
  